In [1]:
import ast
import pandas as pd
from dotenv import load_dotenv
import os
import sys
sys.path.append('../')

import src.geo_functions as gf
import src.flights_functions as ff
import src.divisas_functions as divf

In [2]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic 
from pymongo import MongoClient
import json
from bson.json_util import dumps
import pandas as pd
import re
import unicodedata
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from deep_translator import (MyMemoryTranslator)
import pandas as pd
from pymongo import MongoClient
from pymongo import GEOSPHERE
import json
from bson.json_util import dumps
import ast
from dotenv import load_dotenv
from amadeus import Client, ResponseError
from amadeus import Location

In [3]:
df=gf.df_geonear("Madrid", 800)

/home/lydia/anaconda3/envs/ironhack/lib/python3.8/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


kms=list(df.City)

In [4]:
kms

NameError: name 'kms' is not defined

In [ ]:
#lista_va=[]
#for k in kms:
    #print(k)
    #dest=ff.get_cheapest_price("Madrid",k,"2021-12-18")
    #lista_va.append(dest)

### IATA CODES:

In [ ]:
df=gf.dicc_to_df() 

In [ ]:
df.sample()

In [ ]:
not_spain=df[df["Countries"]!="Espana"]

In [ ]:
spain=df[df["Countries"]=="Espana"]

In [ ]:
not_spain["Cities"]=not_spain.Cities.apply(gf.translator)

In [ ]:
not_spain["Countries"]=not_spain.Countries.apply(gf.translator)

In [ ]:
IATA = pd.concat([not_spain,spain], axis=0)

In [ ]:
IATA.to_csv("../Data/IATAlimpio.csv",index=False)

In [ ]:
IATA=pd.read_csv("../Data/IATAlimpio.csv")

In [ ]:
location=IATA.Cities.apply(gf.get_coordenadas_tipo_point)

In [ ]:
IATA["Type Point"]=location

In [ ]:
IATA["Type Point"]=IATA["Type Point"].apply(gf.string_dicc)

In [ ]:
IATA.head()

In [ ]:
#IATA["Type Point"]=IATA["Type Point"].apply(lambda x: ast.literal_eval(x))

In [ ]:
IATA.columns=["IATA","Country","City","Location"]

In [ ]:
IATA.sample()

In [ ]:
IATA["Country"]=IATA.Country.apply(lambda x: x.lstrip())

In [ ]:
IATA.shape

In [ ]:
len(list(IATA.City.unique()))


In [ ]:
dicc_IATA=IATA.to_dict('records')

In [ ]:
lista_final=list(dicc_IATA)

In [5]:
def insert_Mongo():
    """
    Esta función inserta los datos geográficos en Mongo DB.
    """
    client = MongoClient("localhost:27017")
    db = client.get_database("Tu_viaje_ideal_en_un_click")
    coordenadas = db["coordenadas"]  
    coordenadas.insert_many(lista_final)
 

In [ ]:
insert_Mongo()

In [ ]:
#conexión con Mongo:
client = MongoClient("localhost:27017")
db = client.get_database("Tu_viaje_ideal_en_un_click")
c = db.get_collection("coordenadas")

In [ ]:
#Borro:
c.delete_many({'Location.coordinates': 'nwoknU'})

In [6]:
def get_coordenadas(city):
    """
    Esta función saca las coordenadas de la ciudad que le pases.
    Args: una ciudad (string).
    Return: Las coordeandas de la ciudad que le paso como argumento (latitud y longitud).
    """
    try:
        geolocator = Nominatim(user_agent="Lydia")
        location = geolocator.geocode(query=city, exactly_one=True,timeout=200)
        return location[1]
    except:
        return "Unkown"

In [7]:
get_coordenadas("Madrid")

(40.4167047, -3.7035825)

In [8]:
def geonear(city, radio):
    """
    Esta función devuelve todas las ciudades contenidas dentro del radio que le paso desde el punto de origen especificado.
    Args:city (string).
         radio en kms (int).
    Return: lista de ciudades contenidas en el radio especificado.
    """
    
    client = MongoClient("localhost:27017")
    db = client.get_database("Tu_viaje_ideal_en_un_click")
    coordenadas = db.get_collection("coordenadas")
    coord=get_coordenadas(city)
    query_travel=[{
    "$geoNear": {'near': list(coord[::-1]),
             'distanceField': 'distance',
             'maxDistance': radio,
             'distanceMultiplier': 6371,
             'spherical'  : True}}]

    geoloc_=coordenadas.aggregate(query_travel)
    geoquery_travel=json.loads(dumps(geoloc_))
    return geoquery_travel

In [9]:
def df_geonear(city, radio):
    """
    Esta función sirve para obtener un df en base a la lista de diccionarios obtenida por la función "Geonear".
    Args: city (string).
          radio en kms (int)
    Return: df con todas las ciudades que están dentro de mi radio con sus respectivas distancias.
    """
    geo=geonear(city, radio)
    df= pd.DataFrame(geo)
    df.distance=df.distance.apply([lambda x: int(x)])
    df2=df[df["distance"] < radio]
    df2.drop(columns=["_id"], inplace=True)
    df3=df2.drop(['IATA', 'Country', 'Location'], axis = 1)
    df3.columns=["City", "Distance(kms)"]
    return df3.iloc[1:]

In [10]:
df_geonear(800)

TypeError: df_geonear() missing 1 required positional argument: 'radio'

In [11]:
def get_IATA(city):
    """
    Esta función devuelve el código IATA de un aeropuerto.
    Args: Ciudad (string).
    Return: Código IATA (string de tres letras).
    """
    client = MongoClient("localhost:27017")
    db = client.get_database("Tu_viaje_ideal_en_un_click")
    c = db.get_collection("coordenadas")
    cond={"City": city}
    #pr={"IATA": 1,"Country":0, "_id":0, "City":1, "Location":0}
    IATA=list(c.find(cond))
    IATASS=[]
    IATASS.append(IATA[0]['IATA'])
    return IATA[0]['IATA']

In [12]:
get_IATA("Dubrovnik")

'DBV'

In [13]:
def get_cheapest_price(origen,destino,fecha):
    """
    Esta función me devuelve el precio más bajo para un origen, destino y fecha dadas por el usuario.
    Args:origen = un str de 3 letras (el código IATA del aeropuerto de origen).
         destino = un str de 3 letras (el código IATA del aeropuerto de destino).
         fecha = un str en formato yyyy-mm-dd con la fecha de vuelo.
    Return: El precio más bajo para el vuelo seleccionado.
    
    """
    origen=get_IATA(origen)
    #destino=get_IATA(destino)[0]
    

    load_dotenv()
    amadeus = Client(
        client_id= os.getenv("API_Key"),
        client_secret= os.getenv("API_Secret")
)
    try:
        response = amadeus.shopping.flight_offers_search.get(
            originLocationCode=origen,
            destinationLocationCode=destino,
            departureDate=fecha,
            adults=1)
        #print(response.data)
        list_prices=[]
        for i in range(len(response.data)):
            price=response.data[i]['price']['total']
            list_prices.append(price)

    except ResponseError as error:
        return "There is not any fight for the selected date"
    if len(list_prices)==0:
        return "There is not any fight for the selected date:("
    else: 
        return min(list_prices)


In [81]:
get_cheapest_price("Madrid","ZAZ","2022-02-14")

MAD
ZAZ


'102.63'

In [14]:
city = "Madrid"
city

'Madrid'

In [15]:
radio = 1000

In [16]:
origen=get_IATA(city)

In [17]:
origen

'MAD'

In [18]:
destinos=df_geonear(city, radio)

In [19]:
destinos.head()

,City,Distance(kms)
1,Tambor,11
2,Toledo,68
3,Toledo,68
4,Buenaventura,100
5,Manzanillo,136


In [20]:
lista_destinos = list(destinos.City)
lista_destinos[:10]

['Tambor',
 'Toledo',
 'Toledo',
 'Buenaventura',
 'Manzanillo',
 'Cuenca',
 'Valladolid',
 'Salamanca',
 'Trujillo',
 'Burgos']

In [21]:
IATAS_=[]

for d in lista_destinos:
    iata = get_IATA(d)
    IATAS_.append(iata)


In [22]:
IATAS_[:10]

['TMU', 'TOL', 'TOL', 'BUN', 'ZLO', 'CUE', 'VLL', 'SLM', 'TRU', 'RGS']

In [23]:
precios=[]
for i in IATAS_:
    print(i)
    try:
        precio=get_cheapest_price(city,i,"2022-02-14")
        precios.append(precio)
    except:
        pass

TMU
MAD
TMU
TOL
MAD
TOL
TOL
MAD
TOL
BUN
MAD
BUN
ZLO
MAD
ZLO
CUE
MAD
CUE
VLL
MAD
VLL
SLM
MAD
SLM
TRU
MAD
TRU
RGS
MAD
RGS
BOJ
MAD
BOJ
ZOS
MAD
ZOS
IRJ
MAD
IRJ
RJL
MAD
RJL
ZAZ
MAD
ZAZ
JAE
MAD
JAE
MID
MAD
MID
VIT
MAD
VIT
VIT
MAD
VIT
BGC
MAD
BGC
CDV
MAD
CDV
GRW
MAD
GRW
LUO
MAD
LUO
VLC
MAD
VLC
VLC
MAD
VLC
BJX
MAD
BJX
BJX
MAD
BJX
VRL
MAD
VRL
LGP
MAD
LGP
RMU
MAD
RMU
CDT
MAD
CDT
PNA
MAD
PNA
BIO
MAD
BIO
BUW
MAD
BUW
BJZ
MAD
BJZ
ATM
MAD
ATM
SDR
MAD
SDR
SID
MAD
SID
EAS
MAD
EAS
VSE
MAD
VSE
ALC
MAD
ALC
GRX
MAD
GRX
LOH
MAD
LOH
OVD
MAD
OVD
IXX
MAD
IXX
BIQ
MAD
BIQ
ILD
MAD
ILD
SVQ
MAD
SVQ
EUX
MAD
EUX
SMI
MAD
SMI
EBA
MAD
EBA
LEI
MAD
LEI
REU
MAD
REU
AGP
MAD
AGP
GAN
MAD
GAN
JSY
MAD
JSY
OPO
MAD
OPO
PUF
MAD
PUF
LDE
MAD
LDE
DLI
MAD
DLI
GGF
MAD
GGF
GGF
MAD
GGF
STM
MAD
STM
VGO
MAD
VGO
XRY
MAD
XRY
IBZ
MAD
IBZ
OBI
MAD
OBI
LEU
MAD
LEU
SCQ
MAD
SCQ
CVU
MAD
CVU
GIB
MAD
GIB
LIS
MAD
LIS
BCN
MAD
BCN
BCN
MAD
BCN
LCG
MAD
LCG
JCU
MAD
JCU
VPY
MAD
VPY
CAT
MAD
CAT
FAO
MAD
FAO
TNG
MAD
TNG
PMI
MAD
PMI
TOH
MAD
TOH
TLS
MAD
TLS
PRM


In [24]:
precios

['There is not any fight for the selected date:(',
 '2460.96',
 'There is not any fight for the selected date:(',
 'There is not any fight for the selected date:(',
 '1340.75',
 '332.90',
 'There is not any fight for the selected date:(',
 'There is not any fight for the selected date:(',
 '1001.93',
 '312.29',
 'There is not any fight for the selected date:(',
 '1070.19',
 '2358.86',
 'There is not any fight for the selected date:(',
 '102.63',
 '304.31',
 '1343.73',
 'There is not any fight for the selected date:(',
 'There is not any fight for the selected date:(',
 'There is not any fight for the selected date:(',
 '1645.28',
 '278.19',
 '932.80',
 '103.58',
 '103.58',
 '1340.75',
 '1340.75',
 'There is not any fight for the selected date:(',
 '1424.42',
 'There is not any fight for the selected date:(',
 'There is not any fight for the selected date:(',
 '150.85',
 '124.37',
 'There is not any fight for the selected date:(',
 '145.29',
 'There is not any fight for the selected dat

In [25]:
destinos["Precio vuelo"]=precios

In [26]:
destinos

,City,Distance(kms),Precio vuelo
1,Tambor,11,There is not any fight for the selected date:(
2,Toledo,68,2460.96
3,Toledo,68,There is not any fight for the selected date:(
4,Buenaventura,100,There is not any fight for the selected date:(
5,Manzanillo,136,1340.75
...,...,...,...
144,Brandon,938,1372.50
145,Er-Rachidia,945,There is not any fight for the selected date:(
146,Chambery,968,There is not any fight for the selected date:(
147,El Bayadh,973,There is not any fight for the selected date:(


In [63]:
budget=500

In [ ]:
df["A"] = pd.to_numeric(df["A"], downcast="float")

In [54]:
destinos_filtrados=destinos[destinos["Precio vuelo"]!='There is not any fight for the selected date:(']


In [57]:
diez=destinos_filtrados[:10]

In [60]:
diez["Precio vuelo"]=pd.to_numeric(diez["Precio vuelo"], downcast="float")

<ipython-input-60-b8f1407fe9f2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diez["Precio vuelo"]=pd.to_numeric(diez["Precio vuelo"], downcast="float")


In [68]:
diez["Precio vuelo"]=diez["Precio vuelo"].apply(lambda x:round(x,2))

<ipython-input-68-fba151412502>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diez["Precio vuelo"]=diez["Precio vuelo"].apply(lambda x:round(x,2))


In [69]:
diez.head()

,City,Distance(kms),Precio vuelo
2,Toledo,68,2460.96
5,Manzanillo,136,1340.75
6,Cuenca,138,332.90
9,Trujillo,213,1001.93
10,Burgos,214,312.29


In [70]:
diez=diez[diez["Precio vuelo"]<500]


In [71]:
diez

,City,Distance(kms),Precio vuelo
6,Cuenca,138,332.90
10,Burgos,214,312.29
15,Zaragoza,273,102.63
16,Jaen,274,304.31


In [3]:
div=divf.get_country("Madrid")

In [4]:
div

'Espana'